In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline

In [ ]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

# Load the data_preprocessing.ipynb notebook
with open('data_preprocessing.ipynb', 'r', encoding='utf-8') as f:
    notebook = nbformat.read(f, as_version=4)

# Execute the notebook to make its variables available
ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
ep.preprocess(notebook)


In [ ]:
# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000)

# Fit and transform the training data
X_train_tfidf = tfidf.fit_transform(X_train)

# Transform the testing data
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
# Define the parameter grid
param_grid = {
    'logistic__C': [0.01, 0.1, 1, 10, 100],
    'logistic__penalty': ['l1', 'l2'],
    'logistic__solver': ['liblinear', 'saga']
}

# Create a pipeline with scaling and logistic regression
pipeline = Pipeline([
    ('scaler', MaxAbsScaler()),
    ('logistic', LogisticRegression(max_iter=1000))
])

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the model
grid_search.fit(X_train_tfidf, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Train the model with the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = best_model.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Save the model and vectorizer
joblib.dump(model, 'legal_text_classification_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

In [ ]:
# Load the model and vectorizer
model = joblib.load('legal_text_classification_model.pkl')
tfidf = joblib.load('tfidf_vectorizer.pkl')

# Preprocess new text
new_text = "This is a new legal case about intellectual property."
cleaned_text = preprocess_text(new_text)

# Transform the text using TF-IDF
new_text_tfidf = tfidf.transform([cleaned_text])

# Make a prediction
prediction = model.predict(new_text_tfidf)
print("Predicted Label:", prediction[0])